In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

#### Import compiled_SUPERsnow_flat.nc

In [8]:
fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow_flat.nc'

#ds full
ds = xr.open_dataset(fname,  chunks={'z': 1000000})
ds.close()
ds

<xarray.Dataset>
Dimensions:  (time: 51, z: 298708138)
Coordinates:
  * time     (time) datetime64[ns] 2013-04-03 2013-04-29 ... 2018-05-28
    mask     (z) uint8 dask.array<shape=(298708138,), chunksize=(1000000,)>
    dem      (z) float32 dask.array<shape=(298708138,), chunksize=(1000000,)>
    veg      (z) float32 dask.array<shape=(298708138,), chunksize=(1000000,)>
Dimensions without coordinates: z
Data variables:
    snow     (time, z) int16 dask.array<shape=(51, 298708138), chunksize=(51, 1000000)>